In [12]:
import pandas as pd

dates = pd.read_csv('data/dates.csv')
movies = pd.read_csv('data/movies.csv')
ratings1 = pd.read_csv('data/ratings1.csv')
ratings2 = pd.read_csv('data/ratings2.csv')

#display(movies.info())
#display(ratings1['userId'].nunique())

dates['date'] = pd.to_datetime(dates['date'], dayfirst=True)
dates['year'] = dates['date'].dt.year

#display(dates['year'].value_counts())

ratings = pd.concat(
    [ratings1, ratings2],
    ignore_index=True
)
#display(ratings)

ratings = ratings.drop_duplicates(ignore_index=True)
ratings_dates = pd.concat([ratings, dates], axis=1)
display(ratings_dates.tail(7))
#print('Число строк в таблице ratings: ', ratings.shape[0])

,userId,movieId,rating,date,year
100829,610,164179,5.0,2017-05-03 21:07:11,2017
100830,610,166528,4.0,2017-05-04 06:29:25,2017
100831,610,166534,4.0,2017-05-03 21:53:22,2017
100832,610,168248,5.0,2017-05-03 22:21:31,2017
100833,610,168250,5.0,2017-05-08 19:50:47,2017
100834,610,168252,5.0,2017-05-03 21:19:12,2017
100835,610,170875,3.0,2017-05-03 21:20:15,2017


In [17]:
import pandas as pd
import os as os

def concat_user_files(path):
    filelist = os.listdir(path)
    filelist.sort()
    movie_df = pd.DataFrame({}, columns=['user_id', 'photo_url'])
    for filename in filelist:
        fullpath = path+'/'+filename
        curfile = pd.read_csv(fullpath, sep=';')
        movie_df = pd.concat([movie_df, curfile], ignore_index=True)
    movie_df = movie_df.drop_duplicates(ignore_index=True)
    return(movie_df)
        
display(concat_user_files('users'))

,user_id,photo_url
0,1,abc
1,2,def
2,3,ghi
3,1,abc
4,2,def
5,3,ghi
6,1,abc
7,2,def
8,4,jkl
9,5,mno


In [2]:
import pandas as pd

items_df = pd.DataFrame({
    'item_id': [417283, 849734, 132223, 573943, 19475, 3294095, 382043, 302948, 100132, 312394], 
    'vendor': ['Samsung', 'LG', 'Apple', 'Apple', 'LG', 'Apple', 'Samsung', 'Samsung', 'LG', 'ZTE'],
    'stock_count': [54, 33, 122, 18, 102, 43, 77, 143, 60, 19]
})

purchase_df = pd.DataFrame({
    'purchase_id': [101, 101, 101, 112, 121, 145, 145, 145, 145, 221],
    'item_id': [417283, 849734, 132223, 573943, 19475, 3294095, 382043, 302948, 103845, 100132], 
    'price': [13900, 5330, 38200, 49990, 9890, 33000, 67500, 34500, 89900, 11400]
})

merge_df = items_df.merge(purchase_df, how='inner', on='item_id')

merge_df['amount'] = merge_df['price'] * merge_df['stock_count']

income = merge_df['amount'].sum()

display(income)

19729490

In [87]:
import pandas as pd

import re 
def get_year_release(arg):
    #находим все слова по шаблону "(DDDD)"
    candidates = re.findall(r'\(\d{4}\)', arg) 
    # проверяем число вхождений
    if len(candidates) > 0:
        #если число вхождений больше 0,
	#очищаем строку от знаков "(" и ")"
        year = candidates[0].replace('(', '')
        year = year.replace(')', '')
        return int(year)
    else:
        #если год не указан, возвращаем None
        return 0
    
movies = pd.read_csv('data/ratings_movies.csv')
movies['year release'] = movies['title'].apply(get_year_release)

#display(movies[movies['year release'] == 0].shape[0])
#display(movies[movies['year release'] == 1999].groupby(by=['title'])['rating'].mean().sort_values())
#display(movies[movies['year release'] == 2010].groupby(by=['genres'])['rating'].mean().sort_values())
#display(movies.groupby(by=['userId'])['genres'].nunique().sort_values())

#display(movies.groupby(by=['userId'])['rating'].agg(['count', 'mean']).\
#   sort_values(by=['count', 'mean'], ascending=[True, False]))

#Найдите сочетание жанров (genres) за 2018 год, которое имеет наибольший средний рейтинг
# (среднее по столбцу rating), 
# и при этом число выставленных ему оценок (количество значений в столбце rating) больше 10.

#movies_gr = movies[movies['year release'] == 2018].groupby(by='genres')['rating'].agg(['count', 'mean']).\
#    sort_values(by='mean', ascending=False)

#display(movies_gr[movies_gr['count'] > 10])

movies['year rating'] = movies['date'].apply(lambda x: int(str(x)[:4]))
#display(movies.head())

movies_pt = movies.pivot_table(
    values='rating',
    index='genres',
    columns='year rating',
    aggfunc='mean',
    fill_value=0
)

movies_pt.to_csv('data/result.csv')

In [112]:
import pandas as pd
import numpy as np

orders = pd.read_csv('data/orders.csv', sep=';')
products = pd.read_csv('data/products.csv', sep=';')

orders['Product_ID'] = orders['ID товара']
orders.drop('ID товара', axis=1, inplace=True)

orders_products = orders.merge(products, how='left', on='Product_ID')
#roots[np.isnan(roots)] = 0
#display(orders_products[np.isnan(orders_products['Price'])])
#display(orders_products[orders_products['Отменен'] == 'Да'])

orders_products['Доход'] = orders_products['Количество']*orders_products['Price']
orders_paid = orders_products[orders_products['Оплачен'] == 'Да']

display(orders_paid.groupby(by='ID Покупателя')['Доход'].sum().sort_values(ascending=False))

ID Покупателя
7    17096.0
5    13043.0
8     1344.0
1        0.0
Name: Доход, dtype: float64